# Odczyty uliniowione

W tej części zajmiemy się analiża uliniowionych odczytów (BAM) poprzez platformę Spark. Tym razem zamiast korzystać z API DataFrame posłużymy się językiem SQL.

In [ ]:
! gsutil ls gs://edugen-lab-$USER/bam

## Przygotowanie sesji Spark
Zainicjowanie sesji Spark oraz stworzenie schematu bazy danych z której będziemy korzystać.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

In [ ]:
import os                               # moduł OS języka Python
user_name = os.environ.get('USER')      # pobieramy zmienną środowiskową USER
bucket = f"gs://edugen-lab-{user_name}" # konstruujemy sciezke dostepowa do pliku
print(bucket)

Będziemy korzystać z modułu Spark SQL. Możemy nasze dane traktować jako dane tabelaryczne. Zdefiniujmy tabelę.
Tym razem korzystamy z DataSource -> BAMDataSource

In [ ]:
table_bam =  'alignments'

alignments_path = f"{bucket}/bam/mother10.bam"

spark.sql(f'DROP TABLE IF EXISTS {table_bam}')
spark.sql(f'CREATE TABLE IF NOT EXISTS {table_bam} \
USING org.biodatageeks.sequila.datasources.BAM.BAMDataSource \
OPTIONS(path "{alignments_path}")')


In [ ]:
spark.sql(f"describe {table_bam}").show()

Żeby dostać się do danych w tabeli, bedziemy korzystać z język SQL (standard ANSI). 
Struktura polecen SQL jest nastepująca: 

```sql
SELECT kolumny
FROM tabela
WHERE warunki
GROUP BY wyrazenia grupujace
HAVING warunki na grupy
ORDER BY wyrazenie sortujące
```
Zapytanie może mieć podzapytania zagnieżdzone. W zapytaniu można korzystać ze złączeń z innymi tabeli poprzez JOIN.

# Wybór kolumn (klauzula SELECT)

In [ ]:
spark.sql(f"select * from {table_bam}").show()

In [ ]:
spark.sql(f"select sample_id, contig, pos, mapq from {table_bam}").show()

Pokaż unikalne wartości flag jakie przyjmują wartości z tabeli

In [ ]:
spark.sql(f"select distinct flag from {table_bam} order by flag ASC").show()  # distinct

`Czym (jakim typem danych) są dane zwracane przez spark.sql ("SELECT..")?`

## Sortowanie (klauzula ORDER BY)

In [ ]:
df = spark.sql(f"select distinct flag from {table_bam} order by flag ASC") 
print(type(df))
df.printSchema()

### Filtrowanie wierszy 

In [ ]:
spark.sql(f"select sample_id, contig, pos, mapq from {table_bam} where pos < 3858310").show()

In [ ]:
spark.sql(f"select sample_id, contig, pos, mapq from {table_bam} where pos < 3858310 and flag NOT IN (113,117,121) and mapq > 50").show()

## <span style='background:yellow'> ZADANIE  </span>
Napisz zapytanie które zwróci unikalne wartości CIGAR na contigu 20 na pozycjach 1-1000000. Posortuj względem długości pola cigar malejąco. Jesli kilka CIGAR ma taką samą długość to posortuj rosnąco alfebatycznie. Ile jest takich unikalnych wartości?

In [ ]:
spark.sql(f"select distinct cigar from {table_bam} where pos between 1 and 1000000 and contig= '20' order by length(cigar) DESC, cigar ASC").show()
spark.sql(f"select count (distinct cigar) from {table_bam} where pos between 1 and 1000000 and contig= '20'").show()
spark.sql(f"select distinct cigar from {table_bam} where pos between 1 and 1000000 and contig= '20'").count()

In [ ]:
cnt = spark.sql(f"select distinct cigar from {table_bam} where pos between 1 and 1000000 and contig= '20'").count()

In [ ]:
cnt = spark.sql(f"select count (distinct cigar) from {table_bam} where pos between 1 and 1000000 and contig= '20'").head()[0]

# Grupowanie danych

Pokaż ile jest wierszy o konkretnej wartości flagi

In [ ]:
spark.sql(f"select flag, count(*) as cnt from {table_bam} group by flag order by cnt desc").show()  # AS - alias kolumny

Mozna wykorzystac kilka funkcji agregujących (na roznych kolumnach) w jednym zapytaniu. 

Pokaz ile odczytow ma dana flage. Pokaz takze ich srednia jakosc mapowania oraz minimalna pozycje na ktorej wystepuja

In [ ]:
spark.sql(f"select flag, count(*) as cnt, avg(mapq) as avg_m, min(pos_start) as min from {table_bam} group by flag order by cnt, avg_m desc").show()  # AS - alias kolumny

Funkcje mozemy zagniezdzac. Np: round(avg(kolumna))

In [ ]:
# Uzycie funkcji floor na średniej
spark.sql(f"select flag, count(*) as cnt, floor(avg(mapq)) as avg_m, min(pos_start) as min from {table_bam} group by flag order by cnt, avg_m desc").show()  # AS - alias kolumny

## <span style='background:yellow'> ZADANIE  </span>
Wiedząc, że jako wyrażenie grupujące można wykorzystać wywołanie funkcji na kolumnie napisz zapytanie które policzy ile odczytów ma określoną długość CIGARa. Dla tych grup policz także zaokrągloną (ROUND) wartość jakości mapowania odczytów. Wynik zawęź tylko do odczytów występujących na chromosomie 20 lub 21. Posortuj malejaco liczności grup.

In [ ]:
spark.sql(f"select length(cigar), count(*) as cnt, round (avg(mapq)) as avg_map \
            from {table_bam} \
            where contig IN ('20', '21') \
            group by length(cigar) \
            order by cnt desc").show()  

# Filtrowanie grup danych

Warunki na pojedyncze wiersze nakładamy za pomocą klauzuli WHERE. Jeśli dokonujemy grupowania danych i chcemy w wynikach uzyskać jedynie informacje o grupach, które spełniają określone warunki używamy klauzuli HAVING. (HAVING moze wystapic wylacznie z GROUP BY).

In [ ]:
# Zrob podsumowanie dla grup odczytów o zgodnym flag. Policz srednie mapq oraz minmalna pozycje
# Pokaz jedynie informacja o flagach dla ktorych jest ponad 1000 odczytów.
spark.sql(f"select flag, count(*) as cnt, floor(avg(mapq)) as avg_m, min(pos_start) as min \
            from {table_bam} \
            group by flag \
            having count(*) > 1000 \
            order by cnt, avg_m desc").show()

`Czy w klauzuli HAVING możemy użyc aliasu kolumny cnt?`

Warunki w HAVING też mogą być złożone (złączone AND, OR). Ale warunki w HAVING mogą się odnosić jedynie do kolumn grupujących lub funkcji agregujących

In [ ]:
spark.sql(f"select flag, count(*) as cnt, floor(avg(mapq)) as avg_m, min(pos_start) as min \
            from {table_bam} \
            group by flag \
            having count(*) > 1000 and flag != 99 \
            order by cnt, avg_m desc").show()

In [ ]:
# Bledna konstrukcja - W HAVING mamy warunek na kolumny niegrupujace
# spark.sql(f"select flag, count(*) as cnt, floor(avg(mapq)) as avg_m, min(pos_start) as min \
#             from {table_bam} \
#             group by flag \
#             having count(*) > 1000 and sample_id ='mother' \
#             order by cnt, avg_m desc").show()

## <span style='background:yellow'> ZADANIE  </span>
Wiedząc, że jako wyrażenie grupujące można wykorzystać wywołanie funkcji na kolumnie napisz zapytanie które policzy ile odczytów ma określoną długość CIGARa. 
Dla tych grup policz także zaokrągloną (ROUND) wartość jakości mapowania odczytów. 
Wynik zawęź tylko do odczytów występujących na chromosomie 20 lub 21. Pokaż tylko grupy mające średnia jakosc mapowania powyzej 30. Posortuj malejaco liczności grup.

In [ ]:
spark.sql(f"select length(cigar), count(*) as cnt, round (avg(mapq)) as avg_map \
            from {table_bam} \
            where contig IN ('20', '21') \
            group by length(cigar) \
            having avg(mapq) > 30 \
            order by cnt desc").show()  

## Podzapytania nieskorelowane

Chcemy zobaczyć, jaka jest najwyższa wartość mapq w naszym zbiorze danych.

In [ ]:
spark.sql(f"select max(mapq) from {table_bam} ").show()

Zobaczmy zatem jakie są dane tych odczytów które charakteryzują się najwyższą wartością jakosci mapowania

In [ ]:
spark.sql(f"select sample_id, contig, pos, mapq from {table_bam} where mapq=60 ").show()

Teraz sprobujmy to zapisac bez zaszywania wartości 60 w zapytaniu.

In [ ]:
spark.sql(f"select sample_id, contig, pos, mapq from {table_bam} where mapq=(select max(mapq) from {table_bam}) ").show()

## <span style='background:yellow'> ZADANIE  </span>

1) Sprawdź ile jest odczytów mających maksymalna wartosc mapq. 

2) Zobacz ile jest odczytów ktorych mapq jest w pierwszej 3 wartosci mapq ze zbioru danych.

* podpowiedzi: ograniczenie liczby zwracanych wierszy (LIMIT). 
* przemyślenie operatora porównania wartosci mapq oraz wartosci zwracanych przez podzapytanie

In [ ]:
print(spark.sql(f"select sample_id, contig, pos, mapq from {table_bam} where mapq=(select max(mapq) from {table_bam}) ").count())
spark.sql(f"select distinct mapq from {table_bam} order by mapq DESC LIMIT 5").show()
print(spark.sql(f"select sample_id, contig, pos, mapq from {table_bam} where mapq IN (select distinct mapq from {table_bam} order by mapq DESC LIMIT 5) ").count())


# Wykorzystanie funkcji wbudowanych

Niektóre funkcje do wykorzystania:
ROUND, FLOOR, SIGN, POW, LEAST, LOG
UPPER, LOWER, SUBSTR, 
NVL
CURRENT_DATE
MIN, MAX, SUM, STDDEV, AVG

In [ ]:
spark.sql(f"select distinct sample_id, upper(sample_id), current_date() from {table_bam} ").show()

# Wykorzystanie funkcji rozszerzonych

Istnieją rozszerzenia Sparka do obslugi genomicznych danych. Na przykład pakiet sequila dostarcza metod do rozproszonego wyliczenia glebokosci pokrycia i pileup.

http://biodatageeks.org/sequila/

In [ ]:
# zeby skorzystac z rozszerzen importujemy modul i tworzymy obiekt ss, opakowanie na sesje sparkową
from pysequila import SequilaSession
ss = SequilaSession(spark)

Coverage jest dostępny jako funkcja tabelaryczna, czyli funkcja zwracająca tabelę.

In [ ]:
# konstruujemy zapytanie
cov_query = f"select * from coverage ('{table_bam}', 'mother10', 'blocks')"


In [ ]:
ss.sql(cov_query).show()

## <span style='background:yellow'> ZADANIE  </span>
Pokaż tylko pozycje, które mają głębokość pokrycia większą niż 5.

In [ ]:
ss.sql(f"select * from coverage ('{table_bam}', 'mother10', 'blocks') where coverage >= 5").show()

## <span style='background:yellow'> ZADANIE  </span>
Korzystając z opcji uruchomienia funkcji coverage ktora zwraca pokrycie dla kazdej pozycji niezaleznie (bases zamiast blocks) napisz  zapytanie które zwróci ile pozycji ma daną głębokość pokrycia. Posortuj po liczności malejąco.

In [ ]:
ss.sql(f"select coverage, count(*) as cnt from coverage ('{table_bam}', 'mother10', 'bases') group by coverage order by cnt desc").show()

In [ ]:
ss.stop()

# Wykrywanie wariantów
Na zajeciach skorzystamy z wersji rozproszonej HaplotypeCaller.

In [ ]:
! gatk-hpt-caller-k8s.sh \
  --conf "spark.executor.memory=1g" \
  --conf "spark.driver.memory=1g" \
  --conf "spark.executor.instances=2" \
  --conf "spark.hadoop.fs.gs.block.size=8388608" \
  -R /mnt/data/mapping/ref/ref.fasta \
  -I gs://edugen-lab-$USER/bam/mother10.bam \
  -O gs://edugen-lab-$USER/vcf/mother10.vcf

In [ ]:
! gsutil ls gs://edugen-lab-$USER/vcf/  # sprawdzimy czy plik sie zapisal

In [ ]:
! cat /usr/local/bin/gatk-hpt-caller-k8s.sh  # podejrzymy konfiguracje